In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [3]:
### PANEL GŁÓWNY Numpy ### - Zmienne niezbędne dla wykonania zbioru danych przez Numpy biorą się stąd. 
# Szczegóły słabiej opisanych - zeszyt kursowy 1

observations = 1000  # Ustaw ilość obserwacji (Startowa: 1000)
noic = 10            # Szum w bazie przypadków treningowych.
noit = 1             # Szum wygenerowany dla funkcji testującej (docelowej).
model = [13,7,-12]   # Parametry modelu losowego dla funkcji a*xs + b*xz + c + szum.

#### 1. Tworzenie i składowanie danych

In [4]:
xs = np.random.uniform(low = -noic, high = noic, size =(observations,vnum)) # Gen. zakr. los. zm. wej.(patrz zeszyt 1)
zs = np.random.uniform(-noic,noic,(observations,vnum)) # J.w.- bez opisów bo zbędne. Poz. dla orientacji.

generated_inputs = np.column_stack((xs,zs)) # Składanie z pow. matrycy w formacie observationsx2. (Patrz zeszyt 1)

noise = np.random.uniform(-noit,noit,(observations,1)) # Gen. zakr. los. (Patrz zeszyt 1)

generated_targets = model[0]*xs + model[1]*zs + model[2] + noise # Funkcja symulująca zadane cele. Do niej dąży model. (Patrz zeszyt 1)

np.savez('My_Nums', inputs = generated_inputs, targets = generated_targets) # Tworzy plik My_Nums.npz w tym samym miejscu co ten zeszyt. 
# Zawiera podane matryce w formie ndarray, co dla obsługi TF jest bardzo wygodne.

### 2. Tutaj zaczyna się TensorFlow
Uwaga odnośnie używania TF - obiekty TF nie wykonują się same.
Trzeba im wydać konkretne polecenie. 

Jeśli chcesz na to spojrzeć z innej strony, 
to zeszyt badawczy 2a zawiera opis i przykłady poszczególnych elementów TF.

In [5]:
### Panel Główny TF ### - tutaj będę wyciągał zmienne dla modelu TF,
# żeby było jasne co jest do czego. Normalnie wszystkie zmienne,
# którymi miałbym potencjalnie poźniej manipulować zbieram dla porządku 1 miejscu.

input_size = 2 # Ilość zmiennych wejściowych - tutaj mamy dwie (x,z)
output_size = 1 # Nasza funkcja ma jedno wyjście - y.

### Rysujemy model

In [9]:
# Tworzy podstawnik do którego będziemy podstawiać dane wejściowe z naszego zbioru danych w pliku npz.
# 'float32' to miara precyzji (32bit) - standard dla Pythona to float64, ale tyle na ogół wystarczy.
# W nawiasie kwadratowym znajdują się wymiary podstawianych matryc. 'None' jest w pierwszym nie dlatego,
# że nasze dane nie mają pierwszego wymiaru (tj. ilość rekordów), ale że nie musimy tego określać.
# To przydatne, bo nie trzeba dzięki temu znać dokładnej ilości obserwacji ani pisać odpowiedniego kodu samemu.
# Ważne jedynie, żeby podać ilość zmiennych wejściowych (input_size) i wyjściowych (output_size).
inputs = tf.placeholder(tf.float32,[None,input_size])
# Podstawnik gdzie podamy funkcji dane testowe (cele) - jest w tym samym kształcie co dane wyjściowe,
# więc korzystamy ze zmiennej opisującej ilość zmiennych wyjściowych (output_size).
targets = tf.placeholder(tf.float32,[None,output_size])

### To były podstawniki. Zauważ, że pomiędzy iteracjami nie zachowują one swojej wartości. Są one dla TF
### tymczasową zmienną zewnętrzną z której czerpie dane. Do dalszego działania będziemy potrzebować zmiennych.
### Zwróć uwagę gdzie w TensorFlow są wielkie litery jak na przykład w tf.Variable. To Ci oszczędzi stresów.

# Tworzymy zmienne gdzie model będzie zwracał Wagi (weights) i Obciążenia (biases). Metoda tf.random działa
# w sposób podobny do tego co stworzyliśmy na poprzednim arkuszu. Szczegółowo o niej później.
# Kształty zmiennych są rzecz jasne takie same jak w zeszycie 1, bo wynikają z tej samej logiki.
weights = tf.Variable(tf.random_uniform([input_size,output_size], minval=-0.1, maxval = 0.1))
biases = tf.Variable(tf.random_uniform([output_size], minval=-0.1, maxval = 0.1))

# Tworzymy zmienne wyjściowe. Metoda tf.matmul działa na podobnej zasadzie do użytego w zeszycie 1
# Iloczynu Skalarnego (metoda np.dot - ang. "dot product"), ale jest zgeneralizowana do tensorów.
# Tak samo w nawiasie wstawiamy tam to co chcemy przez siebie przemnożyć.
outputs = tf.matmul(inputs,weights) + biases

### Deklaracja Funkcji Celu*, oraz metody optymalizacji


*in. f. straty [podobnej 'L2' użyliśmy w zeszycie 1] - ang. Objective Function

Więcej w temacie Funkcji Celu pod: https://pl.wikipedia.org/wiki/Funkcja_celowa

In [10]:
# Deklarujemy strukturę funkcji straty dla normy L2 tak jak w Zeszycie 1.
# W TF jest biblioteka takich funkcji 'tf.losses' z której wybieramy
# interesującą nas norma L2 (euklidesowa**) jest tam pod hasłem 'mean_squared_error'.
# Atrybutem pierwszym jest 'labels' reprezentujące klucz prawidłowych odpowiedzi dla uczenia się
# funkcji. Np "1" jeśli na danym "obrazku" widnieje "1" a funkcja ma kilka odpowiedzi do wyboru.
# Drugi atrybut to 'predictions' - logicznie są to dane wyjściowe, czyli wyniki zgadywania naszego modelu.
# Kropka po cyfrze 2 jest tam celowo, żeby na pewno otrzymać w wyniku liczbę typu float.
mean_loss = tf.losses.mean_squared_error(labels=targets, predictions=outputs) / 2. 

** O normach, w tym Euklidesowej: https://pl.wikipedia.org/wiki/Przestrze%C5%84_unormowana

In [11]:
# Deklarujemy metodę optymalizacji (treningu) funkcji. Z zasobów metod 'tf.train' wybieramy tę co ostatnio
# czyli metodę gradientu prostego** (GradientDescentOptimizer), oraz ustalamy tempo uczenia się modelu (learning_rate)
# Do podfunkcji '.minimize' podstawiamy ustaloną wyżej funkcję straty (mean_loss).
optimize = tf.train.GradientDescentOptimizer(learning_rate=0.05).minimize(mean_loss)

** O Metodzie gradientu prostego: https://pl.wikipedia.org/wiki/Metoda_gradientu_prostego

In [13]:
# Kanonicznie 'sess' jest zmienną, którą nazywa się sesję TF. Tak jak w Klasach używa się self. Dla klarowności.
# Obiekty biblioteki tf nie podlegają automatycznemu wykonywaniu wynikającemu z kolejności jak to bywa w Pythonie.
# Jest to użyteczne ze względu na specyfikę uczenia maszynowego, o czym przekonasz się wkrótce.
# Wywołanie sesji jest jednoznaczne z wykonaniem zadań TF. Raz wywołana, sesja jest aktywna do czasu zamknięcia
# Bądź też zamyka się automatycznie, jeśli użyjemy jej w odpowiedniej strukturze (Zeszyt badań 2a)
sess = tf.InteractiveSession()

/home/SpookyProgrammer/anaconda3/envs/tensorenv/lib/python3.6/site-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [14]:
sess.close()